In [1]:
import time
import requests
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import sys
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
import warnings
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('steam.csv')
del df["Unnamed: 0"]
df = df.dropna()
df.isna().sum()#제거됨
df.columns

Index(['매출 순위', '평가', '게임번호'], dtype='object')

In [3]:
id_list = df.게임번호.values
id_list

array(['1361510', '1446780', '1174180', '1824220', '389730', '1245620',
       '1517290', '1202130', '1551360', '1118010', '1569040', '261550',
       '582160', '812140', '582010', '1604030', '1966010', '1506830',
       '1147560', '1991100', '648800', '920210', '1635450',
       '242050,260473,260474,260950', '1533420', '252490', '1966900',
       '1776780', '1158310,1296731,1303182,1303183,1303184', '413150',
       '1277400', '1775390', '323190', '1389360', '108600'], dtype=object)

In [ ]:
# 크롤링
import requests
import time
def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+str(appid), params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()

#https://partner.steamgames.com/doc/store/getreviews 참고    
def get_n_reviews(appid, n=100):#appid는 해당게임의 상점번호
    reviews = []
    cursor = '*'
    params = {
        'json' : 1,
        'filter' : 'all',#필터 없이
        'language' : 'koreana',#한국어로된 리뷰
        'day_range' : 9223372036854775807,#전체기간의 리뷰
        'review_type' : 'all',
        # 'purchase_type' : 'all'
    }

    while n > 0:
        time.sleep(2)#2초씩 쉬면서 크롤링
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']

        if len(response['reviews']) < 100: break

    return reviews

In [ ]:
reviews = []
for i in id_list:
    reviews.append(get_n_reviews(i, 100))# (크롤링할 게임번호, 리뷰 갯수)

In [ ]:

df2 = pd.DataFrame()

for i in range(len(reviews)):
  data = pd.DataFrame.from_dict(reviews[i])
  data['gameid'] = id_list[i]
  df2 = df2.append(data)

df2.reset_index(drop = True)

In [ ]:
df2 = df2[['gameid','review']]
df2

In [ ]:
#전처리 ( 공백제거 따옴표 제거 특문 제거 ,최대문자설정, 좌우공백 제거)
df2['review'] = df2['review'] \
  .replace(r'[^가-힣 ]', ' ', regex=True) \
  .replace("'", '') \
  .replace(r'\s+', ' ', regex=True) \
  .str.strip() \
  .str[:255]

df2.head()

In [ ]:
# 공백 리뷰 제거
df2 = df2[df2['review'].str.strip().astype(bool)]

len(df) # 제거되고난뒤 리뷰 갯수

In [ ]:
df2.rename(columns={'gameid':'게임번호','review': '리뷰'},inplace=True)
df2

In [ ]:
데이터셋 = pd.merge(df,df2 ,how='outer',on = '게임번호')
데이터셋= 데이터셋[['매출 순위','평가','게임번호','리뷰']]

In [ ]:
데이터셋